In [2]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
from scipy.stats import hmean
import matplotlib.pyplot as plt

In [3]:
def csv_parser(log_file, results_file):
    results_list = []
    with open(log_file,"r") as fp:
        for line in fp:
    #         line = fp.readline()
            split = line.split()
            split2 = line.split(",")
            if(len(split)>0):
                if("pmpakos@" in split[0]):
                    matrix_name = ''
#                     print("---------")

            if(len(split)>2):
                if((split[0]=="INFO:") & (split[1]=="loading") & (split[2]=="Mtx")):
                    print(split[-1])
                    matrix_name = split[-1].split("/")[-1]
                    matrix_name = matrix_name.replace(".mtx","")
            if(len(split2)>2):
                if(split2[1]!=" matrix"):
                    line=line.replace("app",matrix_name).strip("DATA_CSV:,")
                    line=line.replace(",","\t")
                    results_list.append(line)
                    print(line)

    file = open(results_file,"w")
    for res in results_list:
        file.write(res)
    file.close()

In [ ]:
log_file = "alveo_log.txt"
results_file = "results.txt"
csv_parser(log_file, results_file)

In [ ]:
log_file = "alveo_log2.txt"
results_file = "results2.txt"
csv_parser(log_file, results_file)

---

In [4]:
def read_results_single(result_file):
    header = ["matrix name","original rows","original cols","original NNZs","padded rows","padded cols","padded NNZs","padding overhead[%]","num of runs","total run time[sec]","time[ms]/run","performance[GFLOPs]","performance (padded)[GFLOPs]","mem_footprint[MB]"]
    df = pd.read_table(result_file, delimiter =",", names=header)
    return df

def read_results(results_list, results_csv):
    dataframes = []
    for result_file in results_list:
        dataframes.append(read_results_single(result_file))
    # get matrices sorted by mem_footprint and then begin processing them (so that I do not have to sort them later...)
    matrices = dataframes[0]["matrix name"].unique()
    # matrices = dataframes[0].groupby(["matrix name","mem_footprint[MB]"]).size().reset_index().sort_values("mem_footprint[MB]").reset_index(drop=True)["matrix name"]
    
    n_runs = len(results_list)
    results_final = []
    for matrix in matrices:
        original_rows = np.asarray(dataframes[0][dataframes[0]["matrix name"]==matrix]["original rows"])[0]
        original_cols = np.asarray(dataframes[0][dataframes[0]["matrix name"]==matrix]["original cols"])[0]
        original_NNZs = np.asarray(dataframes[0][dataframes[0]["matrix name"]==matrix]["original NNZs"])[0]
        padded_rows = np.asarray(dataframes[0][dataframes[0]["matrix name"]==matrix]["padded rows"])[0]
        padded_cols = np.asarray(dataframes[0][dataframes[0]["matrix name"]==matrix]["padded cols"])[0]
        padded_NNZs = np.asarray(dataframes[0][dataframes[0]["matrix name"]==matrix]["padded NNZs"])[0]
        padding_overhead = np.asarray(dataframes[0][dataframes[0]["matrix name"]==matrix]["padding overhead[%]"])[0]
        num_of_runs = np.asarray(dataframes[0][dataframes[0]["matrix name"]==matrix]["num of runs"])[0]
        
        line = matrix + "\t" + str(original_rows) + "\t" + str(original_cols) + "\t" + str(original_NNZs) +\
                        "\t" + str(padded_rows) + "\t" + str(padded_cols) + "\t" + str(padded_NNZs) +\
                        "\t" + str(padding_overhead) + "\t" + str(num_of_runs)
                
        runtime_tot = np.zeros(shape=(n_runs,1))
        runtime_iter = np.zeros(shape=(n_runs,1))
        perf = np.zeros(shape=(n_runs,1))
        perf_padded = np.zeros(shape=(n_runs,1))
        mem_footprint = np.zeros(shape=(n_runs,1))

        for run in range(0,n_runs):
            df = dataframes[run]
            runtime_tot[run] = np.asarray(df[df["matrix name"]==matrix]["total run time[sec]"])
            runtime_iter[run] = np.asarray(df[df["matrix name"]==matrix]["time[ms]/run"])
            perf[run] = np.asarray(df[df["matrix name"]==matrix]["performance[GFLOPs]"])
            perf_padded[run] = np.asarray(df[df["matrix name"]==matrix]["performance (padded)[GFLOPs]"])
            mem_footprint[run] = np.asarray(df[df["matrix name"]==matrix]["mem_footprint[MB]"])
        if(perf[0][0]<0): # why? nobody knows why
            continue

        runtime_tot_hm = hmean(runtime_tot,axis=0)
        runtime_iter_hm = hmean(runtime_iter,axis=0)
        perf_hm = hmean(perf,axis=0)
        perf_padded_hm = hmean(perf_padded,axis=0)
        mem_footprint_hm = hmean(mem_footprint,axis=0)

        line += "\t" + str(runtime_tot_hm[0]) + "\t" + str(runtime_iter_hm[0]) + "\t" + str(perf_hm[0]) + "\t" + str(perf_padded_hm[0]) + "\t" + str(mem_footprint_hm[0]) + "\n"
        results_final.append(line)
        
    file = open(results_csv,"w")
    for line in results_final:
        file.write(line)
    file.close()

def clean_results(start_list, results_list):
    for t1,t2 in zip(start_list,results_list):
        fw = open(t2,"w")
        with open(t1) as fp:
            for line in fp:
                if("DATA_CSV:," in line):
                    fw.write(line.replace("DATA_CSV:,",""))
        fw.close()

In [54]:
start_list = ["/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_1.txt",
              "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_2.txt",
              "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_3.txt",
              "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_4.txt",
              "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_5.txt"]
results_list = ["/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_CLEAN1.txt",
                "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_CLEAN2.txt",
                "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_CLEAN3.txt",
                "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_CLEAN4.txt",
                "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_CLEAN5.txt"]

clean_results(start_list, results_list)
results_csv = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/double_xilinx_spmv_alveo_u280.txt"
read_results(results_list, results_csv)

In [4]:
start_list = ["/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_RCM_1.txt",
              "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_RCM_2.txt",
              "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_RCM_3.txt",
              "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_RCM_4.txt",
              "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_RCM_5.txt"]
results_list = ["/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_RCM_CLEAN1.txt",
                "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_RCM_CLEAN2.txt",
                "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_RCM_CLEAN3.txt",
                "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_RCM_CLEAN4.txt",
                "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_RCM_CLEAN5.txt"]

clean_results(start_list, results_list)
results_csv = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/double_RCM_xilinx_spmv_alveo_u280.txt"
read_results(results_list, results_csv)

In [6]:
start_list = ["/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/dirty/spmv_targeted_matrices_dgal_partitioning_1.txt"]

results_list = ["/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/clean/spmv_targeted_matrices_dgal_partitioning_CLEAN1.txt"]

clean_results(start_list, results_list)
results_csv = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_double_spmv_log_txt/double_dgal_partitioning_xilinx_spmv_alveo_u280.txt"
read_results(results_list, results_csv)

---

---

---

In [ ]:
# 2
import matplotlib.pyplot as plt
nnzs=[95053,106344,115357,136811,144579,144579,191368,193216,193276,193276,196608,228208,259688,259879,294276,307604,329762,335504,342828,352762,380415,390607,394808,394808,394808,394808,394808,394812,394816,395506,401448,408264,424587,424587,446818,458012,474472,496144,539802,540022,587012,629620,695234,725054,818302,874887,954163,958936,971870,1033473,1096948,1173746,1216334,1273389,1358832,1506550,1506810,1511758,1511760,1679599,1793881,2081063,2190591,4000000]
KernelPerf=[1.044785,35.265793,4.87273,15.123922,22.411874,41.888744,25.515733,33.014267,34.280951,6.128742,6.981322,32.873523,25.562358,0.558282,53.003602,50.839435,9.228629,63.796159,10.956297,65.055233,8.38177,32.226971,0.138132,60.688341,58.98379,42.610544,11.755486,64.39602,73.790487,13.672791,10.291691,75.920781,119.517804,65.634101,76.086505,1.096624,84.886305,46.067224,85.784982,0.667418,15.895477,13.83385,109.744909,82.168404,26.700013,139.52428,13.567134,16.670335,14.889197,16.368349,22.722429,180.938184,17.274159,18.49606,44.614768,210.927546,32.685329,389.277198,281.887004,59.809454,49.676857,145.001603,33.487595,18.220412]
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(nnzs, KernelPerf)
ax.set_xlabel('nnz')
ax.set_ylabel('KernelPerf')
ax.set_title('scatter plot')
plt.show()

In [ ]:
log_file = "alveo_log3_timeout.txt"
results_file = "results3_timeout.txt"
csv_parser(log_file, results_file)

In [ ]:
# 3_timeout
import matplotlib.pyplot as plt
nnzs=[10114,10137,10485,10705,10708,10947,11343,20864,21244,12032,12882,12960,13101,13331,13427,26920,14133,14585,14977,14996,15844,15977,16276,16571,17088,17088,18168,18437,18592,19144,19457,19659,19996,20000,20033,20199,20793,21178,21199,21234,21418,25418,21943,22070,22092,22189,22316,22715,23094,23532,23556,23558,23570,23754,24270,24382,24612,24996,24996,47611,47611,25432,26600,26839,27191,55150,55150,55670,55670,27836,28000,29156,29600,30336,31070,31070,31074,32000,32299,32399,32747,32747,32747,32747,32747,33081,34241,34241,34296,34518,35823,36000,37487,37704,38352,38352,38352,38352,38352,39208,39980,39980,39980,39980,39980,40000,40782,81726,43703,44375,45178,46270,48737,49533,99436,99436,100414,52685,53843,54543,54840,56126,56126,112383,57308,59140,63454,66528,66775,69981,70216,71805,72250,74949,75628,76367,76367,76367,76367,76367,76367,77781,80211,83883,88927,94033,97707,98671,99471,105339,106344,115357,117816,125567,129042,138187,138187,138187,138187,138187,138187,138187,138187,138187,138187,138187,144848,149090,159910,207308,219024]
KernelPerf=[0.138501,0.150277,0.155694,0.155758,0.204924,0.097118,0.179963,0.269126,0.242954,0.182728,0.150041,0.183707,0.19551,0.177418,0.234939,0.387868,0.178705,0.216739,0.225528,0.209116,0.174598,0.23784,0.192036,0.165041,0.27734,0.206745,0.217397,0.190121,0.249176,0.297776,0.18974,0.146062,0.227579,0.262123,0.25416,0.260197,0.25546,0.309536,0.313405,0.36887,0.293836,0.255925,0.201933,0.329378,0.358479,0.329109,0.222289,0.275165,0.378057,0.157355,0.375418,0.387177,0.373371,0.425897,0.333702,0.283329,0.325383,0.317827,0.316387,0.522111,0.770155,0.279256,0.424584,0.348292,0.339856,0.751234,0.691436,0.71952,0.695306,0.560882,0.332977,0.464982,0.452088,0.349556,0.285203,0.26633,0.289199,0.412111,0.272646,0.424032,0.486449,0.431463,0.503947,0.431219,0.430989,0.396959,0.497975,0.469199,0.267259,0.333234,0.4825,0.464753,0.532112,0.553575,0.503393,0.571705,0.626232,0.534884,0.570226,0.933279,0.618187,0.655313,0.353194,0.407481,0.550022,0.41945,0.457677,1.420581,0.586448,0.589227,0.721146,0.756231,0.691055,0.833279,0.937806,1.38581,1.378121,0.769911,0.68991,0.728936,0.852281,0.902275,1.046083,1.016843,0.802701,0.700885,1.242102,0.909785,0.886727,0.715808,1.216019,1.219576,1.239907,0.986924,0.870478,1.071532,1.015519,1.275611,0.93029,1.078584,0.941257,0.716816,1.017235,0.752354,1.094628,1.237382,0.900816,0.846304,1.675216,1.342865,1.344323,1.471803,1.535972,1.269501,1.278669,1.733317,1.908634,1.721077,1.471029,1.677454,1.222905,6.461866,1.615259,1.802525,1.624478,1.539645,1.653054,1.646785,1.853975,2.401497,2.844782]
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(nnzs, KernelPerf)
ax.set_xlabel('nnz')
ax.set_ylabel('KernelPerf')
ax.set_title('scatter plot')
plt.show()

In [ ]:
log_file = "alveo_log4_timeout_noreset.txt"
results_file = "results4_timeout_noreset.txt"
csv_parser(log_file, results_file)

In [ ]:
# 4_timeout_noreset
import matplotlib.pyplot as plt
nnzs=[10114,10137,10485,10705,10708,10947,11343,20864,21244,12032,12882,12960,13101,13331,13427,26920,14133,14585,14977,14996,15844,15977,16276,16571,17088,17088,18168,18437,18592,19144,19457,19659,19996,20000,20033,20199,20793,21178,21199,21234,21418,25418,21943,22070,22092,22189,22316,22715,23094,23532,23556,23558,23570,23754,24270,24382,24612,24996,24996,47611,47611,25432,26600,26839,27191,55150,55150,55670,55670,27836,28000,29156,29600,30336,31070,31070,31074,32000,32299,32399,32747,32747,32747,32747,32747,33081,34241,34241,34296,34518,35823,36000,37487,37704,38352,38352,38352,38352,38352,39208,39980,39980,39980,39980,39980,40000,40782,81726,43703,44375,45178,46270,48737,49533,99436,99436,100414,52685,53843,54543,54840,56126,56126,112383,57308,59140,63454,66528,66775,69981,70216,71805,72250,74949,75628,76367,76367,76367,76367,76367,76367,77781,80211,83883,88927,94033,97707,98671,99471,105339,106344,115357,117816,125567,129042,138187,138187,138187,138187,138187,138187,138187,138187,138187,138187,138187,144848,149090,159910,207308,219024]
KernelPerf=[0.153864,0.135795,0.115753,0.16669,0.134919,0.11386,0.15106,0.370106,0.368407,0.196038,0.15135,0.192135,0.123696,0.027922,0.13306,0.491591,0.231518,0.192295,0.177413,0.10573,0.155939,0.227807,0.151905,0.136945,0.223447,0.227702,0.222281,0.27543,0.185077,0.164899,0.264055,0.286945,0.232236,0.187629,0.232489,0.208078,0.258664,0.208761,0.274405,0.250592,0.216012,0.179108,0.199246,0.204328,0.315298,0.253678,0.264303,0.313375,0.259476,0.339852,0.392345,0.32275,0.448073,0.275241,0.285471,0.245538,0.300027,0.208128,0.253454,0.577965,0.488318,0.570116,0.257959,0.272864,0.294488,0.666409,0.638628,0.471874,0.643736,0.387745,0.238757,0.350504,0.287114,0.43928,0.289927,0.408507,0.357929,0.314402,0.356385,0.315607,0.478247,0.472932,0.464823,0.484427,0.316705,0.275032,0.550167,0.519299,0.250756,0.354376,0.533883,0.433518,0.442808,0.620698,0.470037,0.436508,0.349185,0.374076,0.574932,0.552576,0.570853,0.466125,0.513905,0.552298,0.463431,0.345647,0.608006,0.181704,0.534201,0.42998,0.356642,0.667219,0.671771,0.521856,1.131613,1.43986,0.824756,0.486412,0.381724,0.419395,0.770972,0.122273,0.635114,1.01469,0.957975,0.552734,0.78377,0.567916,0.924375,0.899886,0.636306,0.648326,0.552613,0.611869,0.978408,0.94764,0.653572,1.001659,0.827154,0.665673,0.716022,1.108489,0.638635,1.151646,1.003912,1.234021,0.847467,1.101509,1.123331,0.928134,1.173257,1.477828,1.334285,1.352226,1.742763,1.762466,1.717409,1.527595,1.83357,1.792088,1.193871,1.832974,1.665335,1.370305,1.517163,1.177127,1.626336,1.489857,1.70428,2.256757,2.13727]
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(nnzs, KernelPerf)
ax.set_xlabel('nnz')
ax.set_ylabel('KernelPerf')
ax.set_title('scatter plot')
plt.show()

In [ ]:
log_file = "alveo_log5_timeout_noreset_iterations1000.txt"
results_file = "results5_timeout_noreset_iterations1000.txt"
csv_parser(log_file, results_file)

In [ ]:
# 5_timeout_noreset_iter1000
import matplotlib.pyplot as plt
nnzs=[10114,10137,10485,10705,10708,10947,11343,12032,12882,12960,13101,13331,13427,14133,14585,14977,14996,15844,15977,16276,16571,17088,17088,18168,18437,18592,19144,19457,19659,19996,20000,20033,20199,20793,20864,21178,21199,21234,21244,21418,21943,22070,22092,22189,22316,22715,23094,23532,23556,23558,23570,23754,24270,24382,24612,24996,24996,25418,25432,26600,26839,26920,27191,27836,28000,29156,29600,30336,31070,31070,31074,32000,32299,32399,32747,32747,32747,32747,32747,33081,34241,34241,34296,34518,35823,36000,37487,37704,38352,38352,38352,38352,38352,39208,39980,39980,39980,39980,39980,40000,40782,43703,44375,45178,46270,47611,47611,48737,49533,52685,53843,54543,54840,55150,55150,55670,55670,56126,56126,57308,59140,63454,66528,66775,69981,70216,71805,72250,74949,75628,76367,76367,76367,76367,76367,76367,77781,80211,81726,83883,88927,94033,97707,98671,99436,99436,99471,100414,105339,106344,112383,115357,117816,125567,129042,138187,138187,138187,138187,138187,138187,138187,138187,138187,138187,138187,144848,149090,159910,207308,219024]
KernelPerf=[4.91914,5.954771,1.793972,6.134732,4.012776,1.70147,11.649797,8.142452,3.88901,6.574759,6.453167,6.643076,12.282364,4.185208,3.96667,7.158627,3.23783,14.142905,2.031701,4.019644,1.538452,6.965775,7.138463,2.130273,15.655117,5.927611,13.415487,10.545119,9.996524,2.993224,2.837283,3.009728,3.098906,4.110846,14.474419,3.792687,3.198516,6.08863,13.111775,13.304754,8.354345,15.832825,16.527522,13.946671,5.511593,12.227976,9.898906,6.508253,6.546462,6.352886,6.582908,5.732466,14.14818,13.830377,15.244424,2.292249,2.314417,2.229845,7.790464,6.334444,15.859491,15.590839,7.845203,4.327653,2.893958,10.215014,4.305357,13.729346,3.733734,3.742436,3.707917,2.962002,15.219842,8.62327,15.126484,15.44962,14.99891,15.36673,14.715061,3.919805,13.462166,12.775565,1.732771,15.387623,6.467385,2.912407,5.91802,8.061537,12.808689,13.061367,12.846558,12.867996,12.98299,16.422299,9.105177,9.261395,9.186517,9.109197,8.983963,2.952249,17.317683,15.963277,9.300171,8.510297,11.671306,7.057655,9.28054,10.018551,17.6106,16.762596,10.075398,18.02402,10.216633,8.553237,8.642217,8.358341,8.298508,17.642701,20.951446,10.092976,5.315805,17.361875,9.261322,11.203627,10.797767,15.306426,8.438441,12.483696,11.756999,11.86702,11.875209,12.046516,11.90166,12.19156,11.901713,12.10909,12.572598,12.501545,17.54237,13.208679,11.463928,13.694168,10.315908,8.255431,17.753967,17.925007,12.555558,18.096248,11.033825,15.218642,7.281066,15.208989,13.040231,8.009114,16.882659,11.522913,11.545143,11.502228,11.587198,11.502345,11.63103,11.489849,11.544056,11.465047,11.550548,11.50875,12.456859,8.234329,14.363708,10.150352,15.703759]
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(nnzs, KernelPerf)
ax.set_xlabel('nnz')
ax.set_ylabel('KernelPerf')
ax.set_title('scatter plot')
plt.show()

In [ ]:
title = "6_starting60_iterations1000" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

In [ ]:
# 6_starting60_iterations1000
import matplotlib.pyplot as plt
nnzs=[95053,106344,115357,136811,144579,144579,259879,294276,394808,394808,394808,394808,394808,394812,394816,395506,401448,408264,446818,474472,496144,539802,587012,629620,695234,725054,818302,1080044,1173746,1358832,1506550,1506810,1511758,1511760,4000000]
KernelPerf=[4.169409,15.51409,15.584112,7.080192,10.337672,10.336267,9.931511,19.083318,13.889308,13.903714,13.896133,13.885239,13.898965,13.90557,13.943284,13.899874,14.195896,14.326572,14.771396,15.167098,14.986546,15.346862,15.673319,16.103174,16.721872,17.184719,17.422685,22.340221,18.858087,20.090382,19.764153,20.095529,19.86408,19.842303,13.086585]
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(nnzs, KernelPerf)
ax.set_xlabel('nnz')
ax.set_ylabel('KernelPerf')
ax.set_title('scatter plot')
plt.show()

---

In [ ]:
title = ""
log_file = "10"+title
results_file = "results"+title
csv_parser(log_file, results_file)

In [ ]:
title = "7_small_matrices_latency9" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

In [ ]:
title = "8_small_and_large_colnew" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

In [ ]:
title = "9_small_colnew2" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

In [ ]:
title = "10_sparse_matrix_square_block_colnew" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

---

In [5]:
def csv_block_sizes_fixed_x_or_y_parser(results_file, results_file2, dimension):
    header = ["matrix","rows","cols","nnzs","KernelRunTime(ms)","TotalRunTime(ms)", "KernelPerf(GFlops/Sec)","API_Perf(GFlops/Sec)"]
    df = pd.read_table(results_file, delimiter ="\t", names=header)
    df = df.drop(["KernelRunTime(ms)", "TotalRunTime(ms)", "API_Perf(GFlops/Sec)"],axis=1)

    results_list = []
    for i in range(11,16):
        for j in range(4,10):
            size = 2**i
            if(dimension == "x"):
                block_size_y = 2**j
                line = "sparse_"+str(size)+"_block_*_"+str(block_size_y) + "\t" + str(size) + "\t" + str(size) + "\t"+ str(block_size_y)
                
                for k in range(4,10):
                    block_size_x = 2**k
                    matrix_name = "sparse_"+str(size)+"_block_"+str(block_size_x)+"_"+str(block_size_y)+"_x"
                    selection = np.array(df[df["matrix"]==matrix_name])
                    if(len(selection)==0):
                        continue
                    nnzs = selection[0][3]
                    perf = round(selection[0][4],6)
                    
                    line += "\t"+str(perf) + "\t" + str(nnzs)
            elif(dimension == "y"):
                block_size_x = 2**j
                line = "sparse_"+str(size)+"_block_"+str(block_size_x)+"_*" + "\t" + str(size) + "\t" + str(size) + "\t"+ str(block_size_x)

                for k in range(4,10):
                    block_size_y = 2**k
                    matrix_name = "sparse_"+str(size)+"_block_"+str(block_size_x)+"_"+str(block_size_y)+"_y"
                    selection = np.array(df[df["matrix"]==matrix_name])
                    if(len(selection)==0):
                        continue
                    nnzs = selection[0][3]
                    perf = round(selection[0][4],6)
                    
                    line += "\t"+str(perf) + "\t" + str(nnzs)

            line += "\n" 
            if(len(line.split("\t"))==4):
                continue
            results_list.append(line)
            
    file = open(results_file2,"w")
    for res in results_list:
        file.write(res)
    file.close()

In [ ]:
title = "11_sparse_matrix_block_x_full_colnew" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

results_file2= "results_x_"+title
csv_block_sizes_fixed_x_or_y_parser(results_file, results_file2,"x")

In [ ]:
title = "11_sparse_matrix_block_y_full_colnew" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

results_file2= "results_y_"+title
csv_block_sizes_fixed_x_or_y_parser(results_file, results_file2,"y")

In [6]:
def compare_block_matrices(results_file, results_file2):
    header = ["matrix","rows","cols","nnzs","KernelRunTime(ms)","TotalRunTime(ms)", "KernelPerf(GFlops/Sec)","API_Perf(GFlops/Sec)"]
    df = pd.read_table(results_file, delimiter ="\t", names=header)
    df = df.drop(["KernelRunTime(ms)", "TotalRunTime(ms)", "API_Perf(GFlops/Sec)"],axis=1)

    results_list = []
    for i in range(11,16):
        for j in range(4,10):
            for k in range(4,j):
                size = 2**i
                block_size_x = 2**j
                block_size_y = 2**k
                total_nonzeros = int(size/block_size_x * (block_size_x*block_size_y))
                line = "sparse_"+str(size)+"_block_"+str(block_size_x)+"_"+str(block_size_y) + "\t" + str(size) + "\t" + str(size) + "\t"+ str(block_size_x) + "\t"+ str(block_size_y) + "\t" + str(total_nonzeros)
                matrix_name_x = "sparse_"+str(size)+"_block_"+str(block_size_x)+"_"+str(block_size_y)+"_x"
                matrix_name_y = "sparse_"+str(size)+"_block_"+str(block_size_y)+"_"+str(block_size_x)+"_y"        
                selection_x = np.array(df[df["matrix"]==matrix_name_x])
                selection_y = np.array(df[df["matrix"]==matrix_name_y])
                if(len(selection_x)==0):
                    perf_x = 0.0
                else:
                    perf_x = round(selection_x[0][4],6)
                if(len(selection_y)==0):
                    perf_y = 0.0
                else:
                    perf_y = round(selection_y[0][4],6)                
                line += "\t"+str(perf_x) + "\t"+str(perf_y)
                line += "\n" 
                results_list.append(line)
            
    file = open(results_file2,"w")
    for res in results_list:
        file.write(res)
    file.close()

In [ ]:
title = "12_sparse_matrix_block_compare_block_x_y_colnew" + ".txt"
log_file = "alveo_log" + title
results_file = "results" + title
results_file2 = "results2_"+title

csv_parser(log_file, results_file)
compare_block_matrices(results_file, results_file2)

In [ ]:
title = "14_sparse_matrix_square_block_u280" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

In [ ]:
title = "15_sparse_matrix_block_x_u280" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

results_file2= "results_x_"+title
csv_block_sizes_fixed_x_or_y_parser(results_file, results_file2,"x")

In [ ]:
title = "16_sparse_matrix_block_y_u280" + ".txt"
log_file = "alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

results_file2= "results_y_"+title
csv_block_sizes_fixed_x_or_y_parser(results_file, results_file2,"y")

In [ ]:
title = "17_sparse_matrix_block_compare_block_x_y_u280" + ".txt"
log_file = "alveo_log" + title
results_file = "results" + title
results_file2 = "results2_"+title

csv_parser(log_file, results_file)
compare_block_matrices(results_file, results_file2)

---

In [7]:
def csv_block_sizes_random_fixed_x_or_y_parser(results_file, results_file2, regular_title, dimension):
    header = ["matrix","rows","cols","nnzs","KernelRunTime(ms)","TotalRunTime(ms)", "KernelPerf(GFlops/Sec)","API_Perf(GFlops/Sec)"]
    df = pd.read_table(results_file, delimiter ="\t", names=header)
    df = df.drop(["KernelRunTime(ms)", "TotalRunTime(ms)", "API_Perf(GFlops/Sec)"],axis=1)
    
    df_reg = pd.read_table(regular_title, delimiter ="\t", names=header)
    df_reg = df_reg.drop(["KernelRunTime(ms)", "TotalRunTime(ms)", "API_Perf(GFlops/Sec)"],axis=1)

    results_list = []
    for i in range(11,16):
        for j in range(4,10):
            size = 2**i
            if(dimension == "x"):
                block_size_y = 2**j
                for k in range(4,10):
                    block_size_x = 2**k
                    line = "sparse_"+str(size)+"_random_block_"+str(block_size_x)+"_" + str(block_size_y) + "\t" + str(size) + "\t" + str(size) + "\t" + str(block_size_x) + "\t" + str(block_size_y)
                    
                    matrix_name = "sparse_"+str(size)+"_random_block_"+str(block_size_x)+"_"+str(block_size_y)+"_x"
                    selection = np.array(df[df["matrix"].str.contains(matrix_name)])
                    if(len(selection)==0):
                        continue
                    
                    nnzs = selection[0][3]
                    line +=  "\t" + str(nnzs)
                    
                    #regular part
                    matrix_name = "sparse_"+str(size)+"_block_"+str(block_size_x)+"_"+str(block_size_y)+"_x"
                    selection_reg = np.array(df_reg[df_reg["matrix"].str.contains(matrix_name)])
                    perf = round(selection_reg[0][4],6)
                    line += "\t"+str(perf)
                    
                    for l in range(len(selection)):
                        perf = round(selection[l][4],6)
                        line += "\t"+str(perf)
                    
                    line += "\n"
                    if(len(line.split("\t"))==4):
                        continue
                    results_list.append(line)
            elif(dimension == "y"):
                block_size_x = 2**j
                for k in range(4,10):
                    block_size_y = 2**k
                    line = "sparse_"+str(size)+"_random_block_"+str(block_size_x)+"_" + str(block_size_y) + "\t" + str(size) + "\t" + str(size) + "\t" + str(block_size_x) + "\t" + str(block_size_y)
                    
                    matrix_name = "sparse_"+str(size)+"_random_block_"+str(block_size_x)+"_"+str(block_size_y)+"_y"
                    selection = np.array(df[df["matrix"].str.contains(matrix_name)])
                    if(len(selection)==0):
                        continue
                    
                    nnzs = selection[0][3]
                    line +=  "\t" + str(nnzs)
                    
                    #regular part
                    matrix_name = "sparse_"+str(size)+"_block_"+str(block_size_x)+"_"+str(block_size_y)+"_y"
                    selection_reg = np.array(df_reg[df_reg["matrix"].str.contains(matrix_name)])
                    perf = round(selection_reg[0][4],6)
                    line += "\t"+str(perf)
                    
                    for l in range(len(selection)):
                        perf = round(selection[l][4],6)
                        line += "\t"+str(perf)
                    
                    line += "\n"
                    if(len(line.split("\t"))==4):
                        continue
                    results_list.append(line)

    file = open(results_file2,"w")
    for res in results_list:
        file.write(res)
    file.close()

In [8]:
def create_boxplot(data_to_plot, filename, matrix_names, regular_perf, nnz):
    fig = plt.figure(figsize=(15, 7)) # Create a figure instance
    ax = fig.add_subplot(111)         # Create an axes instance
    ax.scatter(matrix_names, regular_perf, s=30)
    
    positions = [i for i in range(0,len(matrix_names))]
    bp = ax.boxplot(data_to_plot, patch_artist=True, positions=positions) # # Create the boxplot. "artist" to get fill color
        
    ax.set_xticklabels(matrix_names,rotation=45)
    plt.title(filename)
    plt.xticks(rotation=90)
    
    for box in bp['boxes']:
        box.set( color='#7570b3', linewidth=2)   # change outline color and linewidth of the boxes
        box.set( facecolor = '#1b9e77' )        # change fill color
    for whisker in bp['whiskers']:
        whisker.set(color='#7570b3', linewidth=2) # change color and linewidth of the whiskers
    for cap in bp['caps']:
        cap.set(color='#7570b3', linewidth=2) # change color and linewidth of the caps
    for median in bp['medians']:
        median.set(color='#b2df8a', linewidth=2)  # change color and linewidth of the medians
    for flier in bp['fliers']:
        flier.set(marker='o', color='#e7298a', alpha=0.5) # change the style of fliers and their fill    
    
    for i, txt in enumerate(nnz):
        ax.annotate(txt, (matrix_names[i], regular_perf[i]), rotation=45)
    # Save the figure
    fig.savefig(filename)
    
def create_boxplots_sparse_random(results_file2):
    header = ["matrix","rows","cols","BS_x","BS_y","nnz","regular","#1","#2","#3","#4","#5"]
    df = pd.read_table(results_file2, delimiter ="\t", names=header)

    # combine these different collections into a list    
    for i in range(11,16):
        size = 2**i
        df_part = df[df["matrix"].str.contains(str(size))]
        if(df_part.size==0):
            continue
        matrix_names = np.asarray(df_part["matrix"])
        matrix_names = [mn.replace("sparse_"+str(size)+"_","").replace("block_","").replace("random_","") for mn in matrix_names]
        regular_perf = np.asarray(df_part["regular"])
        nnz = np.asarray(df_part["nnz"])
        nnz = [str(int(n/1024))+"K" for n in nnz]

        df_part = df_part.drop(["matrix","rows","cols","BS_x","BS_y","nnz","regular"],axis=1).reset_index(drop=True)
        data_to_plot = []
        for index, row in df_part.iterrows():
            # data = np.asarray(df_part.loc[index]) 
            data = np.asarray(df_part.loc[index].fillna(value=df_part.loc[index].mean())) 
            data_to_plot.append(data)
        filename = results_file2.replace("results","boxplot").replace(".txt","_") +str(size) + ".png"
        create_boxplot(data_to_plot, filename, matrix_names, regular_perf, nnz)

In [ ]:
title = "13_sparse_matrix_random_block_x_colnew_u50" + ".txt"
log_file = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_log_txt/alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

results_file2= "results_x_"+title
regular_title = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/results_txt/results11_sparse_matrix_block_x_full_colnew_u50.txt"
csv_block_sizes_random_fixed_x_or_y_parser(results_file, results_file2, regular_title,"x")

create_boxplots_sparse_random(results_file2)

In [ ]:
title = "18_sparse_matrix_random_block_x_u280" + ".txt"
log_file = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_log_txt/alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

results_file2= "results_x_"+title
regular_title = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/results_txt/results15_sparse_matrix_block_x_u280.txt"
csv_block_sizes_random_fixed_x_or_y_parser(results_file, results_file2, regular_title,"x")

create_boxplots_sparse_random(results_file2)

In [ ]:
title = "13_sparse_matrix_random_block_y_colnew_u50" + ".txt"
log_file = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_log_txt/alveo_log"+title
results_file = "results"+title

csv_parser(log_file, results_file)

results_file2= "results_y_"+title
regular_title = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/results_txt/results11_sparse_matrix_block_y_full_colnew_u50.txt"
csv_block_sizes_random_fixed_x_or_y_parser(results_file, results_file2, regular_title,"y")

create_boxplots_sparse_random(results_file2)

In [ ]:
title = "18_sparse_matrix_random_block_y_u280" + ".txt"
log_file = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_log_txt/alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

results_file2= "results_y_"+title
regular_title = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/results_txt/results16_sparse_matrix_block_y_u280.txt"
csv_block_sizes_random_fixed_x_or_y_parser(results_file, results_file2, regular_title,"y")

create_boxplots_sparse_random(results_file2)

---

In [7]:
title = "19_athena_matrices_u280" + ".txt"
log_file = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_log_txt/alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

/various/pmpakos/athena_matrices/helm3d01.mtx
/various/pmpakos/athena_matrices/bundle1.mtx
bundle1	10581	10581	390741	69.096084	69.536290	11.310077	11.238477

/various/pmpakos/athena_matrices/g7jac080sc.mtx
/various/pmpakos/athena_matrices/TSOPF_FS_b162_c1.mtx
/various/pmpakos/athena_matrices/graham1.mtx
/various/pmpakos/athena_matrices/inlet.mtx
/various/pmpakos/athena_matrices/rajat15.mtx
/various/pmpakos/athena_matrices/af23560.mtx
af23560	23560	23560	484256	99.694686	100.228166	9.714781	9.663072

/various/pmpakos/athena_matrices/Pres_Poisson.mtx
/various/pmpakos/athena_matrices/ted_A.mtx
ted_A	10605	10605	424587	58.955075	59.337131	14.403747	14.311005

/various/pmpakos/athena_matrices/olafu.mtx
olafu	16146	16146	515651	89.124952	89.654038	11.571417	11.503129

/various/pmpakos/athena_matrices/FEM_3D_thermal1.mtx
FEM_3D_thermal1	17880	17880	430740	119.183706	119.651650	7.228169	7.199901

/various/pmpakos/athena_matrices/lp1.mtx
/various/pmpakos/athena_matrices/gyro.mtx
/various/pmpak

In [10]:
title = "20_athena_matrices_u50" + ".txt"
log_file = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_log_txt/alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

/various/pmpakos/athena_matrices/helm3d01.mtx
/various/pmpakos/athena_matrices/bundle1.mtx
bundle1	10581	10581	390741	61.267966	61.908520	12.755148	12.623174

/various/pmpakos/athena_matrices/g7jac080sc.mtx
/various/pmpakos/athena_matrices/TSOPF_FS_b162_c1.mtx
/various/pmpakos/athena_matrices/graham1.mtx
/various/pmpakos/athena_matrices/inlet.mtx
/various/pmpakos/athena_matrices/rajat15.mtx
/various/pmpakos/athena_matrices/af23560.mtx
/various/pmpakos/athena_matrices/Pres_Poisson.mtx
/various/pmpakos/athena_matrices/ted_A.mtx
ted_A	10605	10605	424587	52.270340	52.699640	16.245810	16.113469

/various/pmpakos/athena_matrices/olafu.mtx
olafu	16146	16146	515651	79.372408	80.049946	12.993205	12.883232

/various/pmpakos/athena_matrices/FEM_3D_thermal1.mtx
/various/pmpakos/athena_matrices/lp1.mtx
/various/pmpakos/athena_matrices/gyro.mtx
/various/pmpakos/athena_matrices/msc10848.mtx
/various/pmpakos/athena_matrices/small-dense.mtx
small-dense	1000	1000	1000000	73.951557	74.567031	27.044731	26

In [5]:
title = "21_banded_blockdiag_dense_real_u280" + ".txt"
log_file = "/home/pmpakos/metriseis-xilinx-cscmv-alveo-u50/alveo_log_txt/alveo_log"+title
results_file = "results"+title
csv_parser(log_file, results_file)

/various/pmpakos/alveo_matrices/banded/sparse_524288_banded_1.mtx
sparse_524288_banded_1	524288	524288	524288	104.816481	105.422308	1.280502	1.273144

/various/pmpakos/alveo_matrices/banded/sparse_524288_banded_2.mtx
/various/pmpakos/alveo_matrices/banded/sparse_524288_banded_3.mtx
/various/pmpakos/alveo_matrices/banded/sparse_8388608_banded_1.mtx
/various/pmpakos/alveo_matrices/banded/sparse_8388608_banded_2.mtx
/various/pmpakos/alveo_matrices/banded/sparse_8388608_banded_3.mtx
/various/pmpakos/alveo_matrices/block_diagonal/sparse_65536_block_diagonal_4.mtx
sparse_65536_block_diagonal_4	65536	65536	262144	13.337925	13.814025	5.031432	4.858024

/various/pmpakos/alveo_matrices/block_diagonal/sparse_65536_block_diagonal_16.mtx
sparse_65536_block_diagonal_16	65536	65536	1048576	14.019210	18.887773	19.147688	14.212128

/various/pmpakos/alveo_matrices/block_diagonal/sparse_65536_block_diagonal_64.mtx
sparse_65536_block_diagonal_64	65536	65536	4194304	46.225117	46.590009	23.228537	23.046611
